1. Extracción y revisión de tdf_stages_clear

In [50]:
import pandas as pd

#Cargo el archivo tdf_finishers_master.csv y reviso
df_stages = pd.read_csv('../data/raw/tdf_stages_master.csv')

# display(df_stages)
# display(df_stages.info())
# display(df_stages.nunique())

#Titulos de columnas en minusculas
df_stages.columns = df_stages.columns.str.lower()

# display(df_stages)
display(df_stages.info())

<class 'pandas.DataFrame'>
RangeIndex: 2404 entries, 0 to 2403
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   year      2404 non-null   int64
 1   date      2404 non-null   str  
 2   stage     2404 non-null   str  
 3   course    2404 non-null   str  
 4   distance  2404 non-null   str  
 5   type      2404 non-null   str  
 6   winner    2404 non-null   str  
dtypes: int64(1), str(6)
memory usage: 131.6 KB


None

2. Convertimos la columna date a tipo de fecha

In [51]:
df_stages['date'] = pd.to_datetime(df_stages['date'])
display (df_stages)
display(df_stages.info())

,year,date,stage,course,distance,type,winner
0,1903,1903-07-01,1,Paris to Lyon,467 km (290 mi),Plain stage,Maurice Garin (FRA)
1,1903,1903-07-05,2,Lyon to Marseille,374 km (232 mi),Stage with mountain(s),Hippolyte Aucouturier (FRA)
2,1903,1903-07-08,3,Marseille to Toulouse,423 km (263 mi),Plain stage,Hippolyte Aucouturier (FRA)
3,1903,1903-07-12,4,Toulouse to Bordeaux,268 km (167 mi),Plain stage,Charles Laeser (SUI)
4,1903,1903-07-13,5,Bordeaux to Nantes,425 km (264 mi),Plain stage,Maurice Garin (FRA)
...,...,...,...,...,...,...,...
2399,2025,2025-07-23,17,Bollène – Valence,160.4 km (99.7 mi),Etapa llana,Jonathan Milan (ITA)
2400,2025,2025-07-24,18,Vif – Col de la Loze,171.5 km (106.6 mi),Etapa de montaña,Ben O'Connor (AUS)
2401,2025,2025-07-25,19,Albertville – La Plagne,93.1 km (57.8 mi),Etapa de montaña,Thymen Arensman (NED)
2402,2025,2025-07-26,20,Nantua – Pontarlier,184.2 km (114.5 mi),Etapa escarpada,Kaden Groves (AUS)


<class 'pandas.DataFrame'>
RangeIndex: 2404 entries, 0 to 2403
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   year      2404 non-null   int64         
 1   date      2404 non-null   datetime64[us]
 2   stage     2404 non-null   str           
 3   course    2404 non-null   str           
 4   distance  2404 non-null   str           
 5   type      2404 non-null   str           
 6   winner    2404 non-null   str           
dtypes: datetime64[us](1), int64(1), str(5)
memory usage: 131.6 KB


None

3. Limpiamos y transformamos la columna 'distance'

In [52]:
#Vemos que la columna 'distance' tiene el formato 160.4 km (99.7 mi)
#Eliminamos las millas y los caracteres km y convertimos.

def extract_km (distance_str):
    km_index = distance_str.find ('km')
    km_value = distance_str[:km_index].strip()
    return float (km_value)

df_stages['distance_in_km'] = df_stages['distance'].apply(extract_km)
df_stages.drop(columns = 'distance', inplace = True)

display (df_stages)

,year,date,stage,course,type,winner,distance_in_km
0,1903,1903-07-01,1,Paris to Lyon,Plain stage,Maurice Garin (FRA),467.0
1,1903,1903-07-05,2,Lyon to Marseille,Stage with mountain(s),Hippolyte Aucouturier (FRA),374.0
2,1903,1903-07-08,3,Marseille to Toulouse,Plain stage,Hippolyte Aucouturier (FRA),423.0
3,1903,1903-07-12,4,Toulouse to Bordeaux,Plain stage,Charles Laeser (SUI),268.0
4,1903,1903-07-13,5,Bordeaux to Nantes,Plain stage,Maurice Garin (FRA),425.0
...,...,...,...,...,...,...,...
2399,2025,2025-07-23,17,Bollène – Valence,Etapa llana,Jonathan Milan (ITA),160.4
2400,2025,2025-07-24,18,Vif – Col de la Loze,Etapa de montaña,Ben O'Connor (AUS),171.5
2401,2025,2025-07-25,19,Albertville – La Plagne,Etapa de montaña,Thymen Arensman (NED),93.1
2402,2025,2025-07-26,20,Nantua – Pontarlier,Etapa escarpada,Kaden Groves (AUS),184.2


4. Limpiamos columna winner y creamos columna país

In [53]:
# 1. Limpieza inicial (Corchetes y espacios)
df_stages['winner'] = df_stages['winner'].str.replace(r'\[.*?\]', '', regex=True).str.replace(r'\s+', ' ', regex=True).str.strip()

# 2. INTENTO A: País con paréntesis al FINAL -> "Nombre (ESP)"
df_stages[['winner_name', 'country']] = df_stages['winner'].str.extract(r'^(.*)\s+\(([^()]+)\)$')

# 3. INTENTO B: País con paréntesis al PRINCIPIO -> "(ESP) Nombre"
# Rellenamos solo donde 'country' sea nulo
ext_inicio_par = df_stages['winner'].str.extract(r'^\(([^()]+)\)\s+(.*)$')
df_stages['country'] = df_stages['country'].fillna(ext_inicio_par[0])
df_stages['winner_name'] = df_stages['winner_name'].fillna(ext_inicio_par[1])

# 4. INTENTO C: País SIN paréntesis al FINAL -> "Nombre ESP"
# Buscamos 3 letras mayúsculas exactas al final de la línea
ext_fin_sin = df_stages['winner'].str.extract(r'^(.*?)\s+([A-Z]{3})$')
df_stages['country'] = df_stages['country'].fillna(ext_fin_sin[1])
df_stages['winner_name'] = df_stages['winner_name'].fillna(ext_fin_sin[0])

# 5. INTENTO D: País SIN paréntesis al PRINCIPIO -> "ESP Nombre"
# Buscamos 3 letras mayúsculas exactas al principio de la línea
ext_ini_sin = df_stages['winner'].str.extract(r'^([A-Z]{3})\s+(.*)$')
df_stages['country'] = df_stages['country'].fillna(ext_ini_sin[0])
df_stages['winner_name'] = df_stages['winner_name'].fillna(ext_ini_sin[1])

# 6. CASO FINAL: Si no hay país, el nombre es el original
df_stages['winner_name'] = df_stages['winner_name'].fillna(df_stages['winner'])

# Limpieza final de espacios
df_stages['winner_name'] = df_stages['winner_name'].str.strip()
df_stages['country'] = df_stages['country'].str.strip()

#Cmpruebo que no hay nulos en 'winner_name'
# display(df_stages['winner_name'].isnull().sum())
# display(df_stages)

# display(df_stages['country'].isnull().sum())
# display(df_stages[df_stages['winner'] == '—'])
#quedan dos en los que winner '—'. Son etapas sin ganador, por lo que asignamos 'Unknown' a winner_name
df_stages['winner_name'] = df_stages['winner_name'].replace('—', 'cancelled')

# display(df_stages['country'].isnull().sum())
# display(df_stages[df_stages['country'].isnull()])



5. Limpiamos y arreglamos la columna 'country'

In [54]:
#cComprobamos si hay nulos
# display(df_stages['country'].isnull().sum())
# display (df_stages[df_stages['country'].isnull()])

#Comprobamos que muchos de los nulos en la columna es porque el ganador es o bien un pais o un equipo.
#Primer paso, relleno lacolumna con el pais o equipo ganador.
df_stages['country'] = df_stages.apply(lambda row: row['winner_name'] if pd.isnull(row['country']) else row['country'], axis=1)

# display(df_stages['country'].isnull().sum())
# display (df_stages[df_stages['country'].isnull()])
# Comprobamos que ya no hay nulos.

#A través de un diccionario, normalizamos el nombre del pais y eliminamos los campos en los que no es un pais dejandolos en nulo.
# print(df_stages['country'].unique())

country_normalized = {'FRA': 'Francia', 'SUI': 'Suiza', 'LUX': 'Luxemburgo', 'BEL': 'Bélgica', 'ITA': 'Italia', 'ESP': 'España', 'AUT': 'Austria',
                      'GER': 'Alemania', 'NED': 'Países Bajos', 'Switzerland': 'Suiza', 'Netherlands': 'Países Bajos', 'France': 'Francia',
                      'GBR': 'Reino Unido', 'FRG': 'Alemania', 'IRL': 'Irlanda', 'Belgium': 'Bélgica', 'Belgium A': 'Bélgica', 'POR': 'Portugal',
                      'DEN': 'Dinamarca', 'IRE': 'Irlanda', 'AUS': 'Australia', 'COL': 'Colombia', 'USA': 'Estados Unidos', 'NOR': 'Noruega',
                      'CAN': 'Canadá', 'MEX': 'México', 'GDR': 'Alemania', 'URS': 'URSS', 'BRA': 'Brasil', 'UZB': 'Uzbekistán', 'POL': 'Polonia',
                      'SVK': 'Eslovaquia', 'LAT': 'Letonia', 'UKR': 'Ucrania', 'RUS': 'Rusia', 'CZE': 'República Checa', 'SWE': 'Suecia', 'EST': 'Estonia',
                      'KAZ': 'Kazajistán', 'RSA': 'Sudáfrica', 'LTU': 'Lituania', 'SLO': 'Eslovenia', 'ERI': 'Eritrea', 'ECU': 'Ecuador'}
df_stages['country'] = df_stages['country'].map(country_normalized)
# display(df_stages[df_stages['type'] == 'Individual time trial'])

6. Limpiamos y arreglamos columna 'type'

In [55]:
#Comprobamos que no hay si hay nulos
# display (df_stages['type'].isna().sum())

#Comprobamos que valores únicos hay para normalizarlos posteriormente
# print(df_stages['type'].unique())

#Normalizamos los valores mediante un diccionario
type_normalzed = {'Plain stage': 'Flat', 'Stage with mountain(s)': 'Mountain', 'Stage with mountain': 'Mountain', 'Team time trial': 'Team time-trial',
                  'Individual time trial': 'Individual time-trial', 'Mountain time trial': 'Mountain time-trial', 'Stage with mountains': 'Mountain',
                  'Flat stage': 'Flat', 'Medium mountain stage': 'Hilly', 'High mountain stage': 'Mountain', 'Hilly stage': 'Hilly', 'Flat Stage': 'Flat',
                  'Half Stage': 'Double sector', 'Hilly Stage': 'Hilly', 'Mountain Stage': 'Mountain', 'Plain stage with cobblestones': 'Cobblestones',
                  'Mountain stage': 'Mountain', 'Mountain Stage (s)': 'Mountain', 'Intermediate stage': 'Flat', 'Transition stage': 'Hilly',
                  'Flat cobblestone stage': 'Cobblestones',  'Medium mountain stage[c]': 'Hilly', 'Flat': 'Flat', 'Medium-mountain stage': 'Hilly',
                  'Etapa escarpada': 'Hilly', 'Etapa llana': 'Flat', 'Etapa de montaña': 'Mountain', 'Contrarreloj individual': 'Individual time-travel',
                  'Etapa de media montaña': 'Hilly', 'Cronoescalada': 'Mountain time-trial'}

df_stages['type'] = df_stages['type'].map(type_normalzed)
#Compruebo campos de la columna ya normalizados
display (df_stages)
display (df_stages['type'].isna().sum())
print (df_stages['type'].unique())

,year,date,stage,course,type,winner,distance_in_km,winner_name,country
0,1903,1903-07-01,1,Paris to Lyon,Flat,Maurice Garin (FRA),467.0,Maurice Garin,Francia
1,1903,1903-07-05,2,Lyon to Marseille,Mountain,Hippolyte Aucouturier (FRA),374.0,Hippolyte Aucouturier,Francia
2,1903,1903-07-08,3,Marseille to Toulouse,Flat,Hippolyte Aucouturier (FRA),423.0,Hippolyte Aucouturier,Francia
3,1903,1903-07-12,4,Toulouse to Bordeaux,Flat,Charles Laeser (SUI),268.0,Charles Laeser,Suiza
4,1903,1903-07-13,5,Bordeaux to Nantes,Flat,Maurice Garin (FRA),425.0,Maurice Garin,Francia
...,...,...,...,...,...,...,...,...,...
2399,2025,2025-07-23,17,Bollène – Valence,Flat,Jonathan Milan (ITA),160.4,Jonathan Milan,Italia
2400,2025,2025-07-24,18,Vif – Col de la Loze,Mountain,Ben O'Connor (AUS),171.5,Ben O'Connor,Australia
2401,2025,2025-07-25,19,Albertville – La Plagne,Mountain,Thymen Arensman (NED),93.1,Thymen Arensman,Países Bajos
2402,2025,2025-07-26,20,Nantua – Pontarlier,Hilly,Kaden Groves (AUS),184.2,Kaden Groves,Australia


np.int64(0)

<StringArray>
[                  'Flat',               'Mountain',        'Team time-trial',
  'Individual time-trial',    'Mountain time-trial',                  'Hilly',
          'Double sector',           'Cobblestones', 'Individual time-travel']
Length: 9, dtype: str


7. Últimos cambios y exportamos DataFrame

In [56]:
#Elimino la columna winner
df_stages.drop(columns = 'winner', inplace = True)
display (df_stages)

#Limpieza completada. Guardamos
df_stages.to_csv('../data/processed/tdf_stages_clean.csv', index=False)

,year,date,stage,course,type,distance_in_km,winner_name,country
0,1903,1903-07-01,1,Paris to Lyon,Flat,467.0,Maurice Garin,Francia
1,1903,1903-07-05,2,Lyon to Marseille,Mountain,374.0,Hippolyte Aucouturier,Francia
2,1903,1903-07-08,3,Marseille to Toulouse,Flat,423.0,Hippolyte Aucouturier,Francia
3,1903,1903-07-12,4,Toulouse to Bordeaux,Flat,268.0,Charles Laeser,Suiza
4,1903,1903-07-13,5,Bordeaux to Nantes,Flat,425.0,Maurice Garin,Francia
...,...,...,...,...,...,...,...,...
2399,2025,2025-07-23,17,Bollène – Valence,Flat,160.4,Jonathan Milan,Italia
2400,2025,2025-07-24,18,Vif – Col de la Loze,Mountain,171.5,Ben O'Connor,Australia
2401,2025,2025-07-25,19,Albertville – La Plagne,Mountain,93.1,Thymen Arensman,Países Bajos
2402,2025,2025-07-26,20,Nantua – Pontarlier,Hilly,184.2,Kaden Groves,Australia
